In [ ]:
import os, json
import ssl
import tempfile
import kfp

In [ ]:
URL = os.getenv("DKUBE_URL")  # DKube UI URL
token = os.environ.get("DKUBE_USER_ACCESS_TOKEN")  # DKube OAuth token from UI
ssl.CERT_REQUIRED = ssl.CERT_NONE      # workaround if above URL is not signed by a trusted CA

client = kfp.Client(host=URL + "/pipeline", client_id="1", existing_token=token)

In [ ]:
def custom_component():
    import os
    os.system("ls -lR /model")

custom_op = kfp.components.create_component_from_func(custom_component)
storage_op = kfp.components.load_component_from_file("/Users/rajendra/projects/gpuaas/dkube/pipeline/components/storage/component.yaml")

model_name = "mnist-1783"
@kfp.dsl.pipeline(name='StorageOp Example',description='StorageOp Example with custom component')

def storage_test_pipeline():
    with kfp.dsl.ExitHandler(exit_op=storage_op("reclaim", token, namespace="kubeflow",uid="{{workflow.uid}}")):
        input_volumes = json.dumps(["{{workflow.uid}}-model@model://" + model_name])
        storage  = storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes)
        train_op = custom_op().after(storage)
        train_op.add_pvolumes({"/model": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-model")})

In [ ]:
client.create_run_from_pipeline_func(storage_test_pipeline, arguments={})